In [1]:
import numpy as np
import scipy.io
import scipy.linalg
from scipy.linalg import fractional_matrix_power as matpow
from ssid import okid
from control.matlab import * #need for "ss"


In [2]:
# Load data
p = 2   # Number of outputs
q = 2   # Number of inputs
n = 100 # State dimension (Number of structural dofs x 2)
r = 10  # Reduced model order ()
dt = 1  # Timestep
bruntondata = scipy.io.loadmat("./brunton_matlab/brunton_data.mat")
yImpulseFull = bruntondata['yFull']
yImpulse = bruntondata['YY']
uRandom = bruntondata['uRandom']
yRandom = bruntondata['yRandom']
print(yRandom.shape)

(2, 200)


In [3]:
def _svd(*args):
    U,S,V = scipy.linalg.svd(*args, lapack_driver="gesvd")
    return U,S,V.T.conj()

In [4]:
# okid.okid(yRandom,uRandom,dt=1,kmax=200,orm=52,mro=100)

Dimensions
- p = Number of outputs (measured dof or ndof)
- q = Number of inputs (ngdof)
- n = State dimension (2ndof)
- r = Reduced model order (choose)

Full model
- Dimensions of $\mathbf{A}$: n x n
- Dimensions of $\mathbf{B}$: n x q
- Dimensions of $\mathbf{C}$: p x n
- Dimensions of $\mathbf{D}$: p x q

Reduced model
- Dimensions of $\mathbf{\tilde{A}}$: r x r
- Dimensions of $\mathbf{\tilde{B}}$: r x q
- Dimensions of $\mathbf{\tilde{C}}$: p x r
- Dimensions of $\mathbf{\tilde{D}}$: p x q

In [5]:
# example state evolution with unit impulse input

from sympy.matrices import Matrix, eye
B = Matrix(np.array([['a','b','c'],['d','e','f'],['g','h','i']]))
u1 = Matrix([1,0,0])
u2 = Matrix([0,1,0])
u3 = Matrix([0,0,1])
U = eye(3)

Bu1 = Matrix(B*u1).T
Bu2 = Matrix(B*u2).T
Bu3 = Matrix(B*u3).T

display("B = ", B)
display("U = ", U)
display("u1 = ", u1)
display("u2 = ", u2)
display("u3 = ", u3)
display("[Bu1, Bu2, Bu3] = ", Bu1.T.row_join(Bu2.T.row_join(Bu3.T)))
display("BU = ", B*U)

print("this illustrates that the desired impulse response (Y) is the matrix",
      "where each column corresponds to the column vector of output response (yi)",
      "corresponding to an input column vector where the ith element is one",
      "and all others are zero (ui).")

'B = '

Matrix([
[a, b, c],
[d, e, f],
[g, h, i]])

'U = '

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

'u1 = '

...

'[Bu1, Bu2, Bu3] = '

Matrix([
[a, b, c],
[d, e, f],
[g, h, i]])

'BU = '

Matrix([
[a, b, c],
[d, e, f],
[g, h, i]])

this illustrates that the desired impulse response (Y) is the matrix where each column corresponds to the column vector of output response (yi) corresponding to an input column vector where the ith element is one and all others are zero (ui).


In [9]:
# Y = output data: response to unit impulse, or "impulse response," or "markov parameters".
    # dimensions of Y: p x q x nt, where nt = number of timesteps = number of markov parameters = number of blocks
# mc = number of block rows in Hankel matrix
# mo = number of block columns in Hankel matrix
# p = number of outputs
# q = number of inputs
# r = reduced model order = dimension of reduced A = newly assumed dimension of state variable

def era(Y,mo,mc,p,q,r,dt=1):
    # get D from first input_dimension columns of impulse response
    Dr = Y[:,:,0]  # first block of output data
    
    assert Y.shape[:2] == (p,q)  # sanity check that we're passing in the right output data
    assert Y.shape[2] >= mo+mc   # make sure there are enough timesteps to assemble this size of Hankel matrix
    
    # make impulse response into hankel matrix and shifted hankel matrix
    H0 = np.zeros((p*mo, q*mc))
    H1 = np.zeros((p*mo, q*mc))
    for i in range(mo):
        for j in range(mc):
            H0[p*i:p*(i+1), q*j:q*(j+1)] = Y[:,:,i+j+1]
            H1[p*i:p*(i+1), q*j:q*(j+1)] = Y[:,:,i+j+2]

    # reduced svd of hankel matrix
    U,S,V = _svd(H0)
    Sigma = np.diag(S[:r])
    Ur = U[:,:r]
    Vr = V[:,:r]

    # get A from svd and shifted hankel matrix
    Ar = matpow(Sigma,-0.5) @ Ur.T @ H1 @ Vr @ matpow(Sigma,-0.5)

    # get B and C
    Br = (matpow(Sigma,0.5) @ Vr.T)[:,:q]
    Cr = (Ur @ matpow(Sigma,0.5))[:p,:]

    # eigendecomp A
    W,R = scipy.linalg.eig(Ar)

    # get frequencies from eigendecomp of A
    omega = np.log(W + 2*np.pi*i/dt)
    # print(f"{omega=}")

    # get modeshapes from C and eigendecomp of A
    phi = Cr @ R
    # print(f"{phi=}")    

    # return (Ar,Br,Cr,Dr,omega,phi)
    return (Ar,Br,Cr,Dr,S)

In [10]:
def okid(output,input,r):
    pass

In [11]:
## Compute ERA from impulse response
mco = int(np.floor((yImpulse.shape[2]-1)/2)) # m_c = m_o = (m-1)/2
Ar,Br,Cr,Dr,HSVs = era(yImpulse,mco,mco,q,p,r)
sysERA = ss(Ar,Br,Cr,Dr,dt)